In [13]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.base import clone
from sklearn.metrics import balanced_accuracy_score
import pandas as pd
from NRAS import NRAS

In [14]:
def data(file):
    data = pd.read_csv('imbalanced_datasets/%s.csv' %(file), delimiter=',')
    X = np.array(data.iloc[:,:-1])
    y = (np.array(data.iloc[:,-1]))
    return X, y 

In [15]:
datasets = {1:'yeast3',
            2:'yeast1',
            3:'wisconsin',
            4:'vehicle0',
            5:'vehicle1',
            6:'vehicle2',
            7:'vehicle3',
            8:'segment0',
            9:'pima',
            10:'page-blocks0',
            11:'new-thyroid1',
            12:'newthyroid2',
            13:'iris0',
            14:'haberman',
            15:'glass6',
            16:'glass1',
            17:'glass0',
            18:'glass-0-1-2-3_vs_4-5-6',
            19:'ecoli3',
            #20:'ecoli2',
            #21:'ecoli1',
            #22:'ecoli-0_vs_1'
            }

In [16]:
preprocessing = {
	'ros':RandomOverSampler(random_state=1),
	'smote':SMOTE(random_state=1,k_neighbors=5),
	'nras':NRAS(random_state=1,n_neighbors=5),
}

In [17]:
classificator = {
    'tree_clf':DecisionTreeClassifier(random_state=1),
    'svc_clf':SVC(random_state=1),
}

In [18]:
metrics = {
	'balanced_accuracy':balanced_accuracy_score
}

In [19]:
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1)

In [20]:
score=np.zeros((len(preprocessing),len(datasets), n_splits*n_repeats, len(classificator), len(metrics)))

In [21]:
for dataset_id, dataset in enumerate(datasets):
    X, y = data(datasets[dataset])
    
    for fold_id, (train, test) in enumerate(rskf.split(X,y)):
        for prepro_id, prepro in enumerate(preprocessing):         
            for cl_id, cl in enumerate(classificator):
                clf = clone(classificator[cl]) 
                if preprocessing[prepro] == None:
                    X_train, y_train = X[train], y[train]
                else:
                    X_train, y_train = preprocessing[prepro].fit_resample(X[train], y[train])
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X[test])
                
                for metric_id, metric in enumerate(metrics):
                    score[prepro_id, dataset_id, fold_id, cl_id, metric_id] = metrics[metric](y[test],y_pred)
    

2022-05-23 10:02:43,434:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:43,675:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:44,182:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:44,373:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:45,197:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:45,467:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5, 'n_jobs': 1, 'random_state': 1}")
2022-05-23 10:02:46,174:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 't': 0.5

In [22]:
methods=['ROS', 'SMOTE','NRAS']
df_svc = pd.DataFrame(index=range(1,23),columns=['ROS-balanced_accuracy','SMOTE-balanced_accuracy','NRAS'])
df_tree = df_svc.copy()
for cl_id, cl in enumerate(classificator):
    for dataset_id, dataset in  enumerate(datasets):
        for method_id, method in enumerate(methods):
            for metric_id, metric in enumerate(metrics):
                value = np.zeros(n_splits*n_repeats)
                for i in range(n_splits*n_repeats):
                    value[i] = (score[method_id, dataset_id, i, cl_id, metric_id])
                if cl_id==0:
                    df_tree.iloc[dataset_id,metric_id+len(metrics)*method_id]=(value.mean())
                else:
                    df_svc.iloc[dataset_id,metric_id+len(metrics)*method_id]=(value.mean())
                #print(f'{cl}, Dataset: {dataset}, Metoda: {method}, Metryka:{metric} {value.mean()}')


In [23]:
df_tree

,ROS-balanced_accuracy,SMOTE-balanced_accuracy,NRAS
1,0.823674,0.843987,0.807197
2,0.655667,0.655484,0.650828
3,0.92185,0.93568,0.938618
4,0.910455,0.912072,0.90373
5,0.651011,0.651502,0.560382
6,0.899384,0.903399,0.60381
7,0.633472,0.651502,0.5797
8,0.984673,0.987336,0.982273
9,0.681726,0.672598,0.666783
10,0.904533,0.919557,0.898839


In [24]:
df_svc

,ROS-balanced_accuracy,SMOTE-balanced_accuracy,NRAS
1,0.916998,0.916525,0.890436
2,0.717276,0.72266,0.708065
3,0.972952,0.969723,0.969369
4,0.810668,0.808261,0.812215
5,0.520091,0.51772,0.493658
6,0.708984,0.704736,0.560773
7,0.515622,0.51772,0.49784
8,0.985841,0.98622,0.984199
9,0.720881,0.716937,0.718259
10,0.669547,0.684392,0.678821
